<a href="https://colab.research.google.com/github/dean-baldwin/facebook-html-scraper/blob/main/Facebook_Scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Facebook Post Scraper
**Instructions**
1. Go to Facebook `Post` page that you want to scrape.
2. Be sure to click `Read More...` on all the targeted posts.
3. Right click and select `Save As`.
4. Upload it to Colab.
5. Be sure to change the file path and set the output name for the csv.file.


**Install necessary libraries**

In [ ]:
pip install selenium

     |████████████████████████████████| 954 kB 4.3 MB/s 
     |████████████████████████████████| 356 kB 48.4 MB/s 
     |████████████████████████████████| 138 kB 49.7 MB/s 
     |████████████████████████████████| 55 kB 3.4 MB/s 
     |████████████████████████████████| 3.5 MB 27.2 MB/s 
     |████████████████████████████████| 54 kB 3.3 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.7 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
from bs4 import BeautifulSoup as bs
import re as re
import time
import pandas as pd

**USER ACTION: Set the path to the html file**

In [318]:
with open("/content/chfb.html") as fp:
    facebook_page = bs(fp, 'html.parser')

In [319]:
soup = bs(facebook_page.encode("utf-8"), "html")
soup.prettify()

#find parent div for all posts
main = soup.find("div",{"role":"main"})
posts = soup.find("div",{"role":"main"})

In [321]:
company = []
post_date = []
brand_posts = []
post_likes = []
post_comments = []
react_like = []
react_love = []
react_care = []
react_haha = []
react_wow = []
react_sad = []
react_angry = []
post_shares = []
post_commentors = []


for row in posts:
  elements = row.find_all("div",{"style":"border-radius: max(0px, min(8px, ((100vw - 4px) - 100%) * 9999)) / 8px;"})

  for el in elements:    
    comment_cnt = 0
    lk = ''
    lv = ''
    cr = ''
    ha = ''
    ww = ''
    sd = ''
    ag = ''
    pt = ''
    sh = ''
    people = []
    post = el.find("div",{"style":"text-align: start;"})
    links = el.find_all("a")
    attr = el.find_all(attrs={"aria-label":True})

    try:
      if post.text is not None:

        # look for commentor and count number of comments
        for t in el.find_all('div', role=lambda x: x and 'article' in x):
          comment_cnt += 1

          for i in t.find_all('a', role=lambda x: x and 'link' in x):
            aria_label_link = i.attrs
            profile = aria_label_link['href']
            profile = profile.split('?')[0].replace('https://www.facebook.com/','')
            if profile not in people:
              people.append(profile)

        if len(people) == 0:
          post_commentors.append('no comments')
        else:
          post_commentors.append(people)



        # look for shares
        for t in el.find_all('span', dir=lambda x: x and 'auto' in x):
          for s in t(text=re.compile(r' Share')):
            sh = s
        if sh == '':
          post_shares.append('0 Shares')
        else:
          post_shares.append(sh)

        
        # Get reactions
        # Likes
        for i in el.find_all("div", {'aria-label': re.compile(r'^Like:')}):
          aria_label_like = i.attrs
          lk = aria_label_like['aria-label']
        if lk == '':
          react_like.append('Like: 0 people')
        else:
          react_like.append(lk)

        # Love
        for i in el.find_all("div", {'aria-label': re.compile(r'^Love:')}):
          aria_label_love = i.attrs
          lv = aria_label_love['aria-label']
        if lv == '':
          react_love.append('Love: 0 people')
        else:
          react_love.append(lv)

        # Cares
        for i in el.find_all("div", {'aria-label': re.compile(r'^Care:')}):
          aria_label_care = i.attrs
          cr = aria_label_care['aria-label']
        if cr == '':
          react_care.append('Care: 0 people')
        else:
          react_care.append(cr)

        # Haha
        for i in el.find_all("div", {'aria-label': re.compile(r'^Haha:')}):
          aria_label_haha = i.attrs
          ha = aria_label_haha['aria-label']
        if ha == '':
          react_haha.append('Haha: 0 people')
        else:
          react_haha.append(ha)

        # Wow
        for i in el.find_all("div", {'aria-label': re.compile(r'^Wow:')}):
          aria_label_wow = i.attrs
          ww = aria_label_wow['aria-label']
        if ww == '':
          react_wow.append('Wow: 0 people')
        else:
          react_wow.append(ww)

        # Sad
        for i in el.find_all("div", {'aria-label': re.compile(r'^Sad:')}):
          aria_label_sad = i.attrs
          sd = aria_label_sad['aria-label']
        if sd == '':
          react_sad.append('Sad: 0 people')
        else:
          react_sad.append(sd)

        # Angry
        for i in el.find_all("div", {'aria-label': re.compile(r'^Angry:')}):
          aria_label_angry = i.attrs
          ag = aria_label_angry['aria-label']
        if ag == '':
          react_angry.append('Angry: 0 people')
        else:
          react_angry.append(ag)




        pub_attr = links[1].attrs
        publisher = pub_attr['aria-label']

        date_attr = links[3].attrs
        date = date_attr['aria-label']


        likes_attr = attr[4].attrs
        likes = likes_attr['aria-label']

        company.append(publisher)
        post_date.append(date)
        brand_posts.append(post.text.strip())
        post_comments.append(comment_cnt)
        
    except:
      pass


In [323]:
#Stripping non-numeric values
like_count = []
love_count = []
care_count = []
haha_count = []
wow_count = []
sad_count = []
angry_count = []
shares_count = []

for i in react_like:
    s = str(i).replace('Like:','').replace('people','').replace('person','').replace(' ','')
    like_count += [s]
    for i in range(0, len(like_count)):
        like_count[i] = int(like_count[i])

for i in react_love:
    s = str(i).replace('Love:','').replace('people','').replace('person','').replace(' ','')
    love_count += [s]
    for i in range(0, len(love_count)):
        love_count[i] = int(love_count[i])

for i in react_care:
    s = str(i).replace('Care:','').replace('people','').replace('person','').replace(' ','')
    care_count += [s]
    for i in range(0, len(care_count)):
        care_count[i] = int(care_count[i])

for i in react_haha:
    s = str(i).replace('Haha:','').replace('people','').replace('person','').replace(' ','')
    haha_count += [s]
    for i in range(0, len(haha_count)):
        haha_count[i] = int(haha_count[i])

for i in react_wow:
    s = str(i).replace('Wow:','').replace('people','').replace('person','').replace(' ','')
    wow_count += [s]
    for i in range(0, len(wow_count)):
        wow_count[i] = int(wow_count[i])

for i in react_sad:
    s = str(i).replace('Sad:','').replace('people','').replace('person','').replace(' ','')
    sad_count += [s]
    for i in range(0, len(sad_count)):
        sad_count[i] = int(sad_count[i])

for i in react_angry:
    s = str(i).replace('Angry:','').replace('people','').replace('person','').replace(' ','')
    angry_count += [s]
    for i in range(0, len(angry_count)):
        angry_count[i] = int(angry_count[i])

for i in post_shares:
    s = str(i).replace('Shares','').replace('Share','').replace('person','').replace(' ','')
    shares_count += [s]
    for i in range(0, len(shares_count)):
        shares_count[i] = int(shares_count[i])


**USER ACTION: Set the name of the .csv file**

In [324]:
data = {
    "Company": company,
    "Date": post_date,
    "Post": brand_posts,
    "People": post_commentors,
    "Like": like_count,
    "Love": love_count,
    "Care": care_count,
    "Haha": haha_count,
    "Wow": wow_count,
    "Sad": sad_count,
    "Angry": angry_count,
    "Comments": post_comments,
    "Shares": shares_count
}

df = pd.DataFrame(data)


ValueError: ignored

In [315]:
reactions = ['Like','Love','Care','Haha','Wow','Sad','Angry','Comments','Shares']
df['Engagement'] = df[reactions].sum(axis=1)

In [316]:
#Exporting as csv file to program folder
df.to_csv("ch.csv", encoding='utf-8', index=False)